## GPT for code dictation

In [1]:
from transformers import GPT2Tokenizer, DataCollatorForLanguageModeling, TrainingArguments, Trainer, \
                         GPT2LMHeadModel, pipeline
from datasets import Dataset
import pandas as pd

In [35]:
data = pd.read_csv('../data/english_to_latex.csv')

print(data.shape)

data.head(2)

(50, 2)


,English,LaTeX
0,integral from a to b of x squared,"\int_{a}^{b} x^2\,dx"
1,integral from negative 1 to 1 of x squared,"\int_{-1}^{1} x^2\,dx"


In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# avoids a warning in our transformers code
tokenizer.pad_token = tokenizer.eos_token

# Add our prompt to help guide GPT on what to do
CONVERSION_PROMPT = 'English to LaTeX\n\n'  # LaTeX conversion task

CONVERSION_TOKEN = 'LaTeX:'


In [4]:
# This is our "training prompt" that we want GPT2 to recognize and learn
training_examples = f'{CONVERSION_PROMPT}English: ' + data['English'] + '\n' + CONVERSION_TOKEN + ' ' + data['LaTeX'].astype(str)

print(training_examples[0])


English to LaTeX

English: integral from a to b of x squared
LaTeX: \int_{a}^{b} x^2\,dx


## From GPT-3
![](../images/gpt3latex.png)

In [5]:
# At inference time, the input to our model would look like this
print(f"""{CONVERSION_PROMPT}English: integral from a to b of x squared
LaTeX:""")

English to LaTeX

English: integral from a to b of x squared
LaTeX:


In [6]:
task_df = pd.DataFrame({'text': training_examples})

task_df.head(2)

,text
0,English to LaTeX\n\nEnglish: integral from a t...
1,English to LaTeX\n\nEnglish: integral from neg...


In [56]:
print(task_df['text'].loc[0])

English to LaTeX

English: integral from a to b of x squared
LaTeX: \int_{a}^{b} x^2\,dx


In [7]:
latex_data = Dataset.from_pandas(task_df)  # turn a pandas DataFrame into a Dataset

def preprocess(examples):  # tokenize our text but don't pad because our collator will pad for us dynamically
    return tokenizer(examples['text'], truncation=True)

latex_data = latex_data.map(preprocess, batched=True)

latex_data = latex_data.train_test_split(train_size=.7)


  0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
latex_data['train'][0]

{'text': 'English to LaTeX\n\nEnglish: f of x equals integral from 1 to 2 of x over 2\nLaTeX: f(x) = \\int_{1}^{2} \\frac{x}{2}\\,dx',
 'input_ids': [15823,
  284,
  4689,
  49568,
  198,
  198,
  15823,
  25,
  277,
  286,
  2124,
  21767,
  19287,
  422,
  352,
  284,
  362,
  286,
  2124,
  625,
  362,
  198,
  14772,
  49568,
  25,
  277,
  7,
  87,
  8,
  796,
  3467,
  600,
  23330,
  16,
  92,
  36796,
  17,
  92,
  3467,
  31944,
  90,
  87,
  18477,
  17,
  32239,
  11,
  34350],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [9]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
from transformers import AutoModelForCausalLM

In [10]:
# We could also try other Causal models like GPT-J or GPT-Neo
latex_gpt2 = GPT2LMHeadModel.from_pretrained('gpt2')

In [11]:
latex_gpt2

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [12]:
training_args = TrainingArguments(
    output_dir="./english_to_latex",
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=10, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=20,  # batch size for evaluation
    load_best_model_at_end=True,
    logging_steps=5,
    log_level='info',
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(
    model=latex_gpt2,
    args=training_args,
    train_dataset=latex_data["train"],
    eval_dataset=latex_data["test"],
    data_collator=data_collator
)

trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 15
  Batch size = 20


{'eval_loss': 4.727025032043457,
 'eval_runtime': 0.3258,
 'eval_samples_per_second': 46.046,
 'eval_steps_per_second': 3.07}

In [13]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
/opt/homebrew/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 35
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 90


Epoch,Training Loss,Validation Loss
1,3.645900,1.410721
2,1.316900,1.094605
3,0.921500,0.919080
4,0.631100,0.776438
5,0.585700,0.774887
6,0.490600,0.741055
7,0.502900,0.745760
8,0.445000,0.712242
9,0.378300,0.722747
10,0.384800,0.738676


The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 15
  Batch size = 20
Saving model checkpoint to ./english_to_latex/checkpoint-9
Configuration saved in ./english_to_latex/checkpoint-9/config.json
Model weights saved in ./english_to_latex/checkpoint-9/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 15
  Batch size = 20
Saving model checkpoint to ./english_to_latex/checkpoint-18
Configuration saved in ./english_to_latex/checkpoint-18/config.json
Model weights saved in ./english_to_latex/checkpoint-18/pytorch_model.bin
T

TrainOutput(global_step=90, training_loss=0.862057716316647, metrics={'train_runtime': 55.3337, 'train_samples_per_second': 6.325, 'train_steps_per_second': 1.626, 'total_flos': 6606299520000.0, 'train_loss': 0.862057716316647, 'epoch': 10.0})

In [14]:
trainer.evaluate()  # best loss of 0.7122

The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 15
  Batch size = 20


{'eval_loss': 0.7122420072555542,
 'eval_runtime': 0.3168,
 'eval_samples_per_second': 47.347,
 'eval_steps_per_second': 3.156,
 'epoch': 10.0}

In [15]:
# Let's try fine-tuning it again but first let's have the model read a latex tutorial

In [16]:
from bs4 import BeautifulSoup
import requests

latex_tutorial = BeautifulSoup(
    requests.get('https://www.overleaf.com/learn/latex/Learn_LaTeX_in_30_minutes').text, 'html.parser'
).get_text()

with open('../data/latex_tutorial.txt', 'w') as f: 
    f.write(latex_tutorial)


In [17]:
from transformers import TextDataset

calculus_data = TextDataset(
    tokenizer=tokenizer,
    file_path='../data/latex_tutorial.txt',
    block_size=64
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,  # MLM is Masked Language Modelling
)

latex_gpt2 = GPT2LMHeadModel.from_pretrained('gpt2')

training_args = TrainingArguments(
    output_dir="./calculus",
    overwrite_output_dir=True, # overwrite the content of the output directory
    num_train_epochs=1, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    load_best_model_at_end=True,
    logging_steps=1,
    eval_steps=50,
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(
    model=latex_gpt2,
    args=training_args,
    data_collator=data_collator,
    train_dataset=calculus_data.examples[:int(len(calculus_data.examples)*.8)],
    eval_dataset=calculus_data.examples[int(len(calculus_data.examples)*.8):]
)

/opt/homebrew/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Loading features from cached file ../data/cached_lm_GPT2Tokenizer_64_latex_tutorial.txt [took 0.000 s]
loading configuration file config.json from cache at /Users/sinanozdemir/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_he

In [18]:
trainer.evaluate()  # initial loss

***** Running Evaluation *****
  Num examples = 33
  Batch size = 32


{'eval_loss': 4.180856704711914,
 'eval_runtime': 0.949,
 'eval_samples_per_second': 34.774,
 'eval_steps_per_second': 2.108}

In [19]:
trainer.train()

/opt/homebrew/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 129
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 5


Epoch,Training Loss,Validation Loss
1,3.630300,3.863769


***** Running Evaluation *****
  Num examples = 33
  Batch size = 32
Saving model checkpoint to ./calculus/checkpoint-5
Configuration saved in ./calculus/checkpoint-5/config.json
Model weights saved in ./calculus/checkpoint-5/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./calculus/checkpoint-5 (score: 3.863769054412842).


TrainOutput(global_step=5, training_loss=4.008344268798828, metrics={'train_runtime': 14.5156, 'train_samples_per_second': 8.887, 'train_steps_per_second': 0.344, 'total_flos': 4213334016000.0, 'train_loss': 4.008344268798828, 'epoch': 1.0})

In [20]:
trainer.save_model()  #  An ok drop in loss

Saving model checkpoint to ./calculus
Configuration saved in ./calculus/config.json
Model weights saved in ./calculus/pytorch_model.bin


In [21]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 33
  Batch size = 32


{'eval_loss': 3.863769054412842,
 'eval_runtime': 0.842,
 'eval_samples_per_second': 39.192,
 'eval_steps_per_second': 2.375,
 'epoch': 1.0}

In [22]:
calculus_latex_gpt2 = GPT2LMHeadModel.from_pretrained('./calculus')  # load up our gpt pre-trained on calculus

training_args = TrainingArguments(
    output_dir="./calculus_english_to_latex",
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=10, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=20,  # batch size for evaluation
    load_best_model_at_end=True,
    logging_steps=5,
    log_level='info',
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(
    model=calculus_latex_gpt2,
    args=training_args,
    train_dataset=latex_data["train"],
    eval_dataset=latex_data["test"],
    data_collator=data_collator,
)

trainer.evaluate()  # loss is starting slightly lower than before

loading configuration file ./calculus/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "use_cache": true,
  "vocab_size": 50257
}

loading weights fi

{'eval_loss': 4.217632293701172,
 'eval_runtime': 0.3037,
 'eval_samples_per_second': 49.387,
 'eval_steps_per_second': 3.292}

In [23]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
/opt/homebrew/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 35
  Num Epochs = 10
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 90


Epoch,Training Loss,Validation Loss
1,3.157500,1.402207
2,1.309900,1.059423
3,0.917600,0.885223
4,0.631000,0.733441
5,0.558700,0.756555
6,0.460800,0.725905
7,0.464000,0.724250
8,0.418200,0.707149
9,0.370200,0.708756
10,0.364900,0.719450


The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 15
  Batch size = 20
Saving model checkpoint to ./calculus_english_to_latex/checkpoint-9
Configuration saved in ./calculus_english_to_latex/checkpoint-9/config.json
Model weights saved in ./calculus_english_to_latex/checkpoint-9/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 15
  Batch size = 20
Saving model checkpoint to ./calculus_english_to_latex/checkpoint-18
Configuration saved in ./calculus_english_to_latex/checkpoint-18/config.json
Model weights saved in ./calcu

TrainOutput(global_step=90, training_loss=0.8108858439657424, metrics={'train_runtime': 52.218, 'train_samples_per_second': 6.703, 'train_steps_per_second': 1.724, 'total_flos': 6606299520000.0, 'train_loss': 0.8108858439657424, 'epoch': 10.0})

In [24]:
trainer.evaluate()  # pre-training on the tutorial for one epoch led to a minor drop in loss

The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 15
  Batch size = 20


{'eval_loss': 0.7071492075920105,
 'eval_runtime': 0.2892,
 'eval_samples_per_second': 51.87,
 'eval_steps_per_second': 3.458,
 'epoch': 10.0}

In [25]:
trainer.save_model()  # save this model

Saving model checkpoint to ./calculus_english_to_latex
Configuration saved in ./calculus_english_to_latex/config.json
Model weights saved in ./calculus_english_to_latex/pytorch_model.bin


In [26]:
loaded_model = GPT2LMHeadModel.from_pretrained('./calculus_english_to_latex')
latex_generator = pipeline('text-generation', model=loaded_model, tokenizer=tokenizer)

loading configuration file ./calculus_english_to_latex/config.json
Model config GPT2Config {
  "_name_or_path": "./calculus",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.23.1",
  "use_cache": true,
  "vocab_size": 5025

In [27]:
text_sample = 'g of x equals integral from 0 to pi of x'
conversion_text_sample = f'{CONVERSION_PROMPT}English: {text_sample}\n{CONVERSION_TOKEN}'

print(conversion_text_sample)

English to LaTeX

English: g of x equals integral from 0 to pi of x
LaTeX:


In [28]:
print(latex_generator(
    conversion_text_sample, num_beams=5, early_stopping=True, temperature=0.7,
    max_length=len(tokenizer.encode(conversion_text_sample)) + 20
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


English to LaTeX

English: g of x equals integral from 0 to pi of x
LaTeX: g(x) = \int_{0}^{pi} x\,dx\,dx


In [29]:
print(latex_generator(
    conversion_text_sample, num_beams=5, early_stopping=True, temperature=0.7,
    max_length=len(tokenizer.encode(conversion_text_sample)) + 20
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


English to LaTeX

English: g of x equals integral from 0 to pi of x
LaTeX: g(x) = \int_{0}^{pi} x\,dx\,dx


In [60]:
print(text_sample)

r of x is sum from 0 to x of x squared


In [59]:
print(conversion_text_sample)

English to LaTeX

English: r of x is sum from 0 to x of x squared
LaTeX:


In [61]:
print(latex_generator(
    text_sample, 
    num_beams=3, early_stopping=True, temperature=0.7,
    max_length=len(tokenizer.encode(conversion_text_sample)) + 20
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


r of x is sum from 0 to x of x squared

x^2\,dx^2\,dx^2\,dx^2\,dx^2\,dx^2\,


In [30]:
# Another example
text_sample = 'r of x is sum from 0 to x of x squared'
conversion_text_sample = f'{CONVERSION_PROMPT}English: {text_sample}\n{CONVERSION_TOKEN}'

print(latex_generator(
    conversion_text_sample, num_beams=3, early_stopping=True, temperature=0.7,
    max_length=len(tokenizer.encode(conversion_text_sample)) + 20
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


English to LaTeX

English: r of x is sum from 0 to x of x squared
LaTeX: r(x) = \sum_{0}^{x} x^2\,dx^


In [31]:
# Sanity check that a non-finetuned model could not have done this
non_finetuned_latex_generator = pipeline(
    'text-generation', 
    model=GPT2LMHeadModel.from_pretrained('gpt2'),  # not fine-tuned!
    tokenizer=tokenizer
)

loading configuration file config.json from cache at /Users/sinanozdemir/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.23.1

In [32]:
few_shot_prompt = """English to LaTeX

English: f of x is sum from 0 to x of x squared
LaTeX: f(x) = \sum_{0}^{x} x^2 \,dx \
###
English: f of x equals integral from 0 to pi of x to the fourth power
LaTeX: f(x) = \int_{0}^{\pi} x^4 \,dx \
###
English: x squared
LaTeX:"""

In [33]:
print(non_finetuned_latex_generator(
    few_shot_prompt, num_beams=5, early_stopping=True, temperature=0.7,
    max_length=len(tokenizer.encode(few_shot_prompt)) + 20
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


English to LaTeX

English: f of x is sum from 0 to x of x squared
LaTeX: f(x) = \sum_{0}^{x} x^2 \,dx ###
English: f of x equals integral from 0 to pi of x to the fourth power
LaTeX: f(x) = \int_{0}^{\pi} x^4 \,dx ###
English: x squared
LaTeX: f(x) = \sum_{0}^{x} x^2 \,dx ###


In [34]:
print(non_finetuned_latex_generator(
    conversion_text_sample, num_beams=5, early_stopping=True, temperature=0.7,
    max_length=len(tokenizer.encode(conversion_text_sample)) + 20
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


English to LaTeX

English: r of x is sum from 0 to x of x squared
LaTeX: r of x is sum from 0 to x of x squared

LaTeX: r of x
